# Création des modules nécessaires à l'entraînement 


Dans ce notebook, créer les modules comme le dataset, les tokenisers, le processor à partir des audios.

Nous traduisons de l'audio en texte ce qui signifie que nous avons à la fois besoin d'un extracteur de caractéristiques qui traite le signal vocal vers le format d'entrée du modèle, par exemple un vecteur de caractéristiques, et d'un tokenizer qui traite le format de sortie du modèle vers du texte.

## Importation des modules nécessaires


Nous commençons par installer les librairies nécessaires : 
- *datasets* pour créer la base de données 
- *soundfile* pour traiter les audios
- *transformers* pour créer les tokenizer et les extracteurs de decripteurs

In [ ]:
%%capture
!pip install git+https://github.com/huggingface/datasets.git
!pip install soundfile
!pip install transformers==4.11.3

## Création de la base de données



La création de notre base de données se fait avec un fichier csv. Il est composé d'une colonne *file* qui contient le chemin d'accès vers l'audio et d'une colonne *text* qui contient la transcription de l'audio.

In [ ]:
from datasets import load_dataset, load_metric

train_dataset_dir = "/content/data/my_train_file_40.csv"
test_dataset_dir = "/content/data/my_test_file_40.csv"

timit=load_dataset('csv',column_names=['file','text'],skiprows=1,delimiter=",",data_files={'train':train_dataset_dir ,'test': test_dataset_dir})
timit

Using custom data configuration default-87832ac588ceb2ac


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-87832ac588ceb2ac/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['file', 'text'],
        num_rows: 936
    })
    test: Dataset({
        features: ['file', 'text'],
        num_rows: 40
    })
})

In [ ]:
output_dir = "/content/data/data_40" #A changer si on veut créer une nouvelle base de données
timit

DatasetDict({
    train: Dataset({
        features: ['file', 'text'],
        num_rows: 936
    })
    test: Dataset({
        features: ['file', 'text'],
        num_rows: 40
    })
})

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=1):
    assert num_examples <= len(dataset), "On vérifie que le nombre d'exemples à afficher et inférieur au nombres de données"
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

On affiche aléatoirement un certains nombres d'exemples de transcriptions d'audios.

In [ ]:
show_random_elements(timit["train"].remove_columns(["file"]), num_examples=5)

,text
0,zv 977 mv
1,mi 561 hb
2,qv 372 gx
3,ug 931 od
4,by 050 jj


Les transcriptions peuvent contenir certains caractères spéciaux, tels que - Sans modèle de langue, il est beaucoup plus difficile de classer les morceaux de discours en fonction de ces caractères spéciaux, car ils ne correspondent pas vraiment à une unité sonore caractéristique. Par exemple, la lettre "s" a un son plus ou moins clair, alors que le caractère spécial "." n'en a pas. Aussi, pour comprendre le sens d'un signal vocal, il n'est généralement pas nécessaire d'inclure les caractères spéciaux dans la transcription.

Nous avons également supprimé les caractères spéciaux " " et ' dans ce cas spécial des plaques d'immatriculation.

Le modèle n'a pas besoin apprendre à prédire quand un mot se termine, ce n'est pas génant que ce soit toujours une séquence de caractères.

En outre, nous normalisons le texte pour qu'il ne comporte que des lettres minuscules.

In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    return batch

timit = timit.map(remove_special_characters)

Parameter 'function'=<function remove_special_characters at 0x7fdeadae7c20> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/936 [00:00<?, ?ex/s]

  0%|          | 0/40 [00:00<?, ?ex/s]

Regardons les données pré-traitées

In [ ]:
show_random_elements(timit["train"].remove_columns(["file"]),5)

,text
0,fg 863 mu
1,qw 491 ej
2,wc 426 me
3,yv 512 qz
4,ca 644 hq


Notre jeu de données contient le chemin d'accès aux audios.

## Création des tokenizer

En CTC, il est courant de classer les morceaux de discours en lettres, nous allons donc faire de même ici. Extrayons toutes les lettres distinctes des données d'entraînement et de test et construisons notre vocabulaire à partir de cet ensemble de lettres.

Nous écrivons une fonction de mappage qui concatène toutes les transcriptions en une longue transcription, puis transforme la chaîne en un ensemble de caractères. Il est important de passer l'argument batched=True à la fonction map(...) afin que la fonction de mappage ait accès à toutes les transcriptions en même temps.

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocabs = timit.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=timit.column_names["train"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Maintenant, nous créons l'union de toutes les lettres distinctes dans l'ensemble de données d'entraînement et l'ensemble de données de test et convertissons la liste résultante en un dictionnaire énuméré.

In [ ]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 16,
 '0': 25,
 '1': 14,
 '2': 0,
 '3': 19,
 '4': 9,
 '5': 8,
 '6': 1,
 '7': 24,
 '8': 22,
 '9': 36,
 'a': 11,
 'b': 35,
 'c': 3,
 'd': 10,
 'e': 2,
 'f': 13,
 'g': 27,
 'h': 32,
 'i': 18,
 'j': 31,
 'k': 23,
 'l': 33,
 'm': 15,
 'n': 20,
 'o': 29,
 'p': 28,
 'q': 5,
 'r': 30,
 's': 26,
 't': 34,
 'u': 6,
 'v': 7,
 'w': 21,
 'x': 12,
 'y': 17,
 'z': 4}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

Enfin, nous ajoutons également un jeton de remplissage qui correspond au token blanc de CTC. Le token blanc est un composant essentiel de l'algorithme CTC pour éviter les répétitions.

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)
vocab_dict

{'0': 25,
 '1': 14,
 '2': 0,
 '3': 19,
 '4': 9,
 '5': 8,
 '6': 1,
 '7': 24,
 '8': 22,
 '9': 36,
 '[PAD]': 38,
 '[UNK]': 37,
 'a': 11,
 'b': 35,
 'c': 3,
 'd': 10,
 'e': 2,
 'f': 13,
 'g': 27,
 'h': 32,
 'i': 18,
 'j': 31,
 'k': 23,
 'l': 33,
 'm': 15,
 'n': 20,
 'o': 29,
 'p': 28,
 'q': 5,
 'r': 30,
 's': 26,
 't': 34,
 'u': 6,
 'v': 7,
 'w': 21,
 'x': 12,
 'y': 17,
 'z': 4,
 '|': 16}

In [ ]:
import json
import os
output_dir_vocab = output_dir + '/vocab'

if not os.path.exists(output_dir_vocab):
 os.makedirs(output_dir_vocab)

with open(output_dir_vocab+'/vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

Nous utilisons le fichier json pour instancier un objet de la classe Wav2Vec2CTCTokenizer

In [ ]:
from transformers.utils.dummy_tokenizers_objects import T5TokenizerFast
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer(output_dir_vocab+'/vocab.json', unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
T5TokenizerFast

transformers.utils.dummy_tokenizers_objects.T5TokenizerFast

## Création des extracteurs de descripteurs

La parole est un signal continu et pour être traitée par des ordinateurs, elle doit d'abord être discrétisée, ce qui est généralement appelé **échantillonnage**. Le taux d'échantillonnage joue un rôle important dans la mesure où il définit le nombre de points de données du signal vocal mesurés par seconde. Par conséquent, un échantillonnage avec un taux d'échantillonnage plus élevé permet une meilleure approximation du signal vocal réel, mais nécessite également plus de valeurs par seconde.

Un point de contrôle pré-entraîné s'attend à ce que ses données d'entrée aient été échantillonnées plus ou moins à partir de la même distribution que les données sur lesquelles il a été entraîné. Les mêmes signaux vocaux échantillonnés à deux taux différents ont une distribution très différente, par exemple, si l'on double le taux d'échantillonnage, les points de données sont deux fois plus longs. Ainsi, avant d'affiner un point de contrôle pré-entraîné d'un modèle ASR, il est crucial de vérifier que le taux d'échantillonnage des données utilisées pour pré-entraîner le modèle correspond au taux d'échantillonnage de l'ensemble de données utilisé pour affiner le modèle.

Wav2Vec2 a été pré-entraîné sur les données audio de LibriSpeech et LibriVox qui étaient toutes deux échantillonnées à **16kHz**. Nous devons donc d'abord augmenter ou réduire l'échantillonnage du signal vocal pour le faire correspondre à la fréquence d'échantillonnage des données utilisées pour le pré-entraînement.

Un objet extracteur de caractéristiques Wav2Vec2 nécessite les paramètres suivants pour être instancié :

- ` feature_size` : Les modèles de parole prennent en entrée une séquence de vecteurs de caractéristiques. Si la longueur de cette séquence varie évidemment, la taille des caractéristiques ne doit pas varier. Dans le cas de Wav2Vec2, la taille de la caractéristique est 1 parce que le modèle a été entraîné sur le signal vocal brut.

- `sampling_rate `: Le taux d'échantillonnage sur lequel le modèle est entraîné.

- `padding_value` : Pour l'inférence par lots, les entrées plus courtes doivent être complétées par une valeur spécifique.

- `do_normalize` : Si l'entrée doit être normalisée à la moyenne zéro et à la variance unitaire ou non. En général, les modèles vocaux sont plus performants lorsque l'entrée est normalisée.

- `return_attention_mask` : Si le modèle doit utiliser un masque d'attention pour l'inférence par lots. En général, les modèles devraient **toujours utiliser** l'attention_mask pour masquer les tokens paddés. Cependant, en raison d'un choix de conception très spécifique du point de contrôle "base" de Wav2Vec2, de meilleurs résultats sont obtenus en n'utilisant aucun attention_mask. Ceci n'est pas recommandé pour les autres modèles de parole. Pour plus d'informations, vous pouvez consulter ce numéro. Important Si vous voulez utiliser ce cahier pour affiner le réglage de large-lv60, ce paramètre doit être réglé sur True.

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

Pour rendre l'utilisation de Wav2Vec2 aussi conviviale que possible, l'extracteur de caractéristiques et le tokenizer sont intégrés dans une seule classe Wav2Vec2Processor, de sorte qu'il suffit d'avoir un modèle et un objet processeur.

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)


Nous enregistrons le processor.


In [ ]:
output_dir_processor = output_dir + '/Processor'
processor.save_pretrained("/content/data/data_40/Processor")

Maitenant on peut traiter la base de données.

## Preprocess Data

Notre jeu de données contient le chemin d'accès aux audios.

In [ ]:
timit["train"][0]

{'file': '/content/drive/MyDrive/Data/Audio/Audio_Yanis/ov-318-vx.wav',
 'text': 'ov 318 vx'}

Wav2Vec2 attend l'entrée dans le format d'un tableau à 1 une dimension de 16 kHz. Cela signifie que le fichier audio doit être chargé et rééchantillonné.

Nous utilisons librosa qui va à partir du chemin d'accès, rééchantilloné l'audio à 16kHz et le transformer en un tableau à 1 dimension.

In [ ]:

import librosa

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate  = librosa.load(batch["file"], sr=16000)
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["text"]
    return batch

In [ ]:
timit = timit.map(speech_file_to_array_fn, remove_columns=timit.column_names["train"], num_proc=4)
timit

#0:   0%|          | 0/234 [00:00<?, ?ex/s]

#1:   0%|          | 0/234 [00:00<?, ?ex/s]

#2:   0%|          | 0/234 [00:00<?, ?ex/s]

#3:   0%|          | 0/234 [00:00<?, ?ex/s]

#0:   0%|          | 0/10 [00:00<?, ?ex/s]

#1:   0%|          | 0/10 [00:00<?, ?ex/s]

#2:   0%|          | 0/10 [00:00<?, ?ex/s]

#3:   0%|          | 0/10 [00:00<?, ?ex/s]

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/cor

DatasetDict({
    train: Dataset({
        features: ['speech', 'sampling_rate', 'target_text'],
        num_rows: 936
    })
    test: Dataset({
        features: ['speech', 'sampling_rate', 'target_text'],
        num_rows: 40
    })
})

In [ ]:
timit.save_to_disk(output_dir + '/dataset')

Ecoutons quelques fichiers audio pour mieux comprendre l'ensemble de données et vérifier que l'audio a été correctement chargé.

In [ ]:
import IPython.display as ipd
import numpy as np
import random
rand_int = random.randint(0, len(timit["train"])-1)

ipd.Audio(data=np.asarray(timit["train"][rand_int]["speech"]), autoplay=True, rate=16000)

Vérifions une dernière fois que les données sont correctement préparées, en imprimant la forme de l'entrée vocale, sa transcription et le taux d'échantillonnage correspondant.

In [ ]:
import numpy as np
rand_int = random.randint(0, len(timit["train"])-1)

print("Target text:", timit["train"][rand_int]["target_text"])
print("Input array shape:", np.asarray(timit["train"][rand_int]["speech"]).shape)
print("Sampling rate:", timit["train"][rand_int]["sampling_rate"])

Target text: ex 339 py
Input array shape: (96000,)
Sampling rate: 16000


Bien ! Tout semble correct - les données sont un tableau unidimensionnel, la fréquence d'échantillonnage correspond toujours à 16kHz, et le texte cible est normalisé.

Enfin, nous pouvons traiter l'ensemble de données au format attendu par le modèle pour l'entraînement. Nous allons utiliser la fonction` map(...)`.

Premièrement, nous chargeons et rééchantillonnons les données audio, en appelant simplement ` batch["audio"]`. Ensuite, nous extrayons les valeurs d'entrée du fichier audio chargé. Dans notre cas, le Wav2Vec2Processor ne fait que normaliser les données. Troisièmement, nous encodons les transcriptions en identifiants d'étiquettes.

In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

Nous appliquons cette fonctions à toutes les données

In [ ]:
timit_prepared = timit.map(prepare_dataset, remove_columns=timit.column_names["train"], batch_size=8, num_proc=4, batched=True)

#1:   0%|          | 0/30 [00:00<?, ?ba/s]

#0:   0%|          | 0/30 [00:00<?, ?ba/s]

#2:   0%|          | 0/30 [00:00<?, ?ba/s]

#3:   0%|          | 0/30 [00:00<?, ?ba/s]

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:158: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:158: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:158: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths o

Il ne reste plus qu'à le sauvegarder.

In [ ]:
timit_prepared.save_to_disk(output_dir + '/dataset_prepared')

In [ ]:
timit_prepared

DatasetDict({
    train: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 936
    })
    test: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 40
    })
})